In [1]:
// hacky way to set java path since macos breaks this
val userhome = System.getProperty("user.home")
val jniPath = s"${userhome}/software/z3/build"
val newPath = Array(jniPath) ++  System.getProperty("java.library.path").split(":")
System.setProperty("java.library.path",newPath.distinct.mkString(":"))
//set sys_paths to null so that java.library.path will be reevaluated next time it is needed
val sysPathsField = classOf[ClassLoader].getDeclaredField("sys_paths");
sysPathsField.setAccessible(true);
sysPathsField.set(null, null);
// note: make sim link between bounder and Bounder due to macos case sensitivity before hand
val path = s"${userhome}/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
//unzip -p jdiff.jar META-INF/MANIFEST.MF
interp.load.cp(os.Path(path))

import $ivy.`org.plotly-scala:plotly-almond_2.13:0.8.2`
import $ivy.`com.github.pathikrit::better-files:3.9.1`

import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import plotly._, element._, layout._, Plotly._
import ujson.Value
import sys.process._


userhome: String = "/home/s"
jniPath: String = "/home/s/software/z3/build"
newPath: Array[String] = Array(
  "/home/s/software/z3/build",
  "",
  "/home/s/software/z3/build",
  "/home/s/software/z3/build",
  "/usr/java/packages/lib/amd64",
  "/usr/lib/x86_64-linux-gnu/jni",
  "/lib/x86_64-linux-gnu",
  "/usr/lib/x86_64-linux-gnu",
  "/usr/lib/jni",
  "/lib",
  "/usr/lib"
)
res0_3: String = ":/home/s/software/z3/build:/home/s/software/z3/build:/usr/java/packages/lib/amd64:/usr/lib/x86_64-linux-gnu/jni:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu:/usr/lib/jni:/lib:/usr/lib"
sysPathsField: java.lang.reflect.Field = private static java.lang.String[] java.lang.ClassLoader.sys_paths
path: String = "/home/s/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                          

import $ivy.$                                         


import $ivy.$                             
// import $ivy.`edu.colorado.plv.bounder:soot

In [2]:
import better.files._

import scala.util.Random
import edu.colorado.plv.bounder.{Driver,PickleSpec,Action,RunConfig}
import upickle.default.read
import upickle.default.write
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
import edu.colorado.plv.bounder.{ExperimentsDb,BounderUtil}
import scala.concurrent.duration._
import scala.language.postfixOps
import slick.driver.H2Driver.api._
import slick.jdbc.{GetResult,SQLActionBuilder}
import scala.concurrent.Await
import edu.colorado.plv.bounder.lifestate.{SpecSpace,LifecycleSpec, SDialog, SAsyncTask,FragmentGetActivityNullSpec}


Driver.setZ3Path(s"${userhome}/software/z3/build")

var android_home_possible = List(s"${userhome}/Library/Android/sdk", s"${userhome}/Android/Sdk")
var android_home = android_home_possible.find(p => File(p).exists()).get             
BounderUtil.setEnv(Map("DYLD_LIBRARY_PATH" -> s"${userhome}/software/z3/build","ANDROID_HOME" -> android_home,"HOME" -> userhome))
System.setProperty("user.dir", s"${System.getProperty("user.home")}/Documents/source/bounder/notebooks/ossExp/data");
val expDir = File(System.getProperty("user.dir"))
val baseDir = File(System.getProperty("user.home")) / "Documents/data/historia_generalizability"

java.library.path set to: /home/s/software/z3/build::/usr/java/packages/lib/amd64:/usr/lib/x86_64-linux-gnu/jni:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu:/usr/lib/jni:/lib:/usr/lib


import better.files._


import scala.util.Random

import edu.colorado.plv.bounder.{Driver,PickleSpec,Action,RunConfig}

import upickle.default.read

import upickle.default.write

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}

import edu.colorado.plv.bounder.{ExperimentsDb,BounderUtil}

import scala.concurrent.duration._

import scala.language.postfixOps

import slick.driver.H2Driver.api._

import slick.jdbc.{GetResult,SQLActionBuilder}

import scala.concurrent.Await

import edu.colorado.plv.bounder.lifestate.{SpecSpace,LifecycleSpec, SDialog, SAsyncTask,FragmentGetActivityNullSpec}



android_home_possible: List[String] = List(
  "/home/s/Library/Android/sdk",
  "/home/s/Android/Sdk"
)
android_home: String = "/home/s/Android/Sdk"
res1_17: String = "/home/s/Documents/source/Bounder/notebooks/ossExp"
expDir: File = /home/s/Documents/source/bounder/notebooks/ossExp/data
baseDir: File = /home/s/Documents/data/historia_generalizability

In [3]:
val benchmarkList = "benchmark-list.txt"
object ExpConfig{
    val excludeFromProguardCheck = List("ch.blinkenlights.android.vanilla", "com.darshancomputing.BatteryIndicatorPro") // apps that break the tool I am using for proguard check but don't use proguard (checked manually)
    def apkAndFilt(file:File) = {
        val path = file.toString
        val spliton = if(path.contains("fdroid")) "fdroid" else "oss"
        val pkg = path.split(spliton)(1).split("/")
        s"${path}:${pkg.tail.head}"
    }
    val apkList = baseDir.glob("**/*.apk").toList.filter{pathF =>
        val path = pathF.toString // filter out the duplicate APKs for connectbot and antennapod
        !(path.contains("de.danoeh.antennapod") && path.contains("app-play")) && !(path.contains("org.connectbot") && path.contains("app-google"))}
        /*moved to another dir, comment out for speed*///.filter{a => excludeFromProguardCheck.exists(x => a.toString.contains(x)) || !Driver.detectProguard(a.toString)} // remove apps that use proguard (we can't distinguish app and framework)
    assert(apkList.exists(_.toString.contains("org.connectbot")))
    assert(apkList.exists(_.toString.contains("de.danoeh.antennapod")))
    val benchmarkListFile = File(benchmarkList)
    benchmarkListFile.overwrite(apkList.map(apkAndFilt(_)).mkString("\n"))
    
}
ExpConfig.apkList.size

benchmarkList: String = "benchmark-list.txt"
defined object ExpConfig
res2_2: Int = 62

In [4]:
// Apps that use proguard (should be empty, moved to another folder)
val useProguard = baseDir.glob("**/*.apk").toList.filter{pathF =>
        val path = pathF.toString // filter out the duplicate APKs for connectbot and antennapod
        !(path.contains("de.danoeh.antennapod") && path.contains("app-play")) && !(path.contains("org.connectbot") && path.contains("app-google"))}.filter{ pathF => !ExpConfig.apkList.exists(o => o == pathF)}

useProguard: List[File] = List()

In [ ]:
// Create json files for config runs
def createConfigsFromList():Unit = {
    val disallowSpecs = new SpecSpace(Set(), disallowSpecs = Set(
        FragmentGetActivityNullSpec.getActivityNull,
        SAsyncTask.disallowDoubleExecute,
        SDialog.disallowDismiss
        ))
    List(Driver.FindCallins, Driver.MakeSensitiveDerefFieldCaused, Driver.MakeSensitiveDerefCallinCaused).foreach{ mode =>
        val baseConfig = RunConfig(limit = -1)
        case class ApkSample(apk:String, pkg:String, outDir:File)
        val apkSamples = (expDir / ".." / benchmarkList)
            .contentAsString
            .split("\n").toList.map{v => 
                val pkg = v.split(":")(1)
                // println(s"package: $pkg")
                val apk = v.split(":")(0)
                val apkDirCfg = apk.toString.replace(baseDir.toString,"${baseDir}")
                val outDirCfg = "${baseDirOut}/" + pkg
                val outDir = expDir / pkg
    //             val getSample = outDir / "getSample.json"
    //             if(getSample.exists())getSample.delete()
                // val samples = if(pkg.contains("lyonbros")) 90 else 200
                val cfg = baseConfig.copy(apkPath=apkDirCfg, outFolder=Some(outDirCfg), specSet = PickleSpec.mk(disallowSpecs)) // 5 dereference samples per class
    //             getSample.append(write[RunConfig](cfg))
                //outDir.createIfNotExists(true)
                // sample dereference locations
                // SampleDeref for choosing subset of samples, MakeAllDeref for getting all of them
                Action(mode, baseDirOut=Some(expDir.toString), baseDirApk = Some(baseDir.toString), // b u options
                      cfg, filter=Some(pkg))
            }
        apkSamples.foreach{action => 
            //println(s"action: ${action}")
            println(s"out folder: ${action.getOutFolder}")
            println(action.filter.get)
            val f = File(s"data/${action.filter.get}.cfg")
            f.overwrite(write[RunConfig](action.config))
            Driver.runAction(action)
        }
    }
}
createConfigsFromList()

out folder: /home/s/Documents/source/bounder/notebooks/ossExp/data/org.connectbot
org.connectbot
java.library.path set to: /home/s/software/z3/build::/usr/java/packages/lib/amd64:/usr/lib/x86_64-linux-gnu/jni:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu:/usr/lib/jni:/lib:/usr/lib
out folder: /home/s/Documents/source/bounder/notebooks/ossExp/data/de.danoeh.antennapod
de.danoeh.antennapod
java.library.path set to: /home/s/software/z3/build::/usr/java/packages/lib/amd64:/usr/lib/x86_64-linux-gnu/jni:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu:/usr/lib/jni:/lib:/usr/lib


In [ ]:
// val f = File("data/alljson_whydoesthisincrease.txt")
// val existing = f.contentAsString.split('\n')
// val globed = expDir.glob("**/*.json").toList

// globed.find{g => !existing.contains(g.toString.split("/data/")(1))}
object CountEach{
    val cfgs = expDir.glob("**/*.cfg").map(f => read[RunConfig](f.contentAsString())).toSet
    
}
CountEach.cfgs.groupBy{c => 
    val dspecs = c.specSet.getDisallowSpecSet
    assert(dspecs.size == 1)
    (dspecs.head.target.identitySignature,c.apkPath)}.map{case (g,v) => (g,v.size)}

In [ ]:
// Utilities for manipulating run configs

def splitConfig(cfg:File, maxJobsPerFile:Int) = {
    val cfgContent = read[RunConfig](cfg.contentAsString)
    val parentDir = cfg.parent
    val cfgName = cfg.name.split('.')(0)
    assert(cfg.name.split('.')(1) == "json", "Input should have ext json")
    val newCfg = cfgContent.initialQuery.grouped(maxJobsPerFile).zipWithIndex
    
    newCfg.foreach{
        case (iqs,ind) => 
            val outfname = s"${cfgName}_${ind}.json"
            println(s"index: ${ind} size: ${cfg.size} fname: ${outfname}")
            val outf = parentDir / outfname
            outf.overwrite(write[RunConfig](cfgContent.copy(initialQuery=iqs)))
    }
}

// update timeout in run config
def setTimeout(nt:Int) = {
    val configs = expDir.glob("**/*.json").toList
    assert(configs.forall(f => !f.toString.contains(".ipynb_checkpoints")), "Delete dumb jupyter backup file")
    configs.foreach{cfgf =>
        val cfg = read[RunConfig](cfgf.contentAsString)
        cfgf.delete()
        cfgf.overwrite(write[RunConfig](cfg.copy(timeLimit=nt)))
    }
//     write[RunConfig](.map{c => read[RunConfig](c.contentAsString)}.toList.head.copy(timeLimit = 180))
}
//splitConfig(File("/Users/shawnmeier/Documents/source/bounder/experiments/5fdroid5dev_sampderef/com.lyonbros/sample.json"),5)

val db = new ExperimentsDb(Some(s"${userhome}/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"))
case class Count(n:Int)
case class StrRes(s:String)
implicit val getCountResult = GetResult(r => Count(r.<<))
implicit val getStrResult = GetResult(r => StrRes(r.<<))

def pauseJobs() = {
    val pauseJobsQ = sql"""update jobs set status='paused' where status='new';"""
    Await.result(db.db.run(pauseJobsQ.as[StrRes]), 30 seconds)
}
def resumeJobs() = {
    val pauseJobsQ = sql"""update jobs set status='new' where status='paused';"""
    Await.result(db.db.run(pauseJobsQ.as[StrRes]), 30 seconds)
}
def clearInputs() = {
    val clearInputsQ = sql"""delete from apks where apkname like 'jar_%' or apkname like 'spec_%';"""
    Await.result(db.db.run(clearInputsQ.as[StrRes]), 30 seconds)
    val clearInputsTableQ = sql"""delete from inputs;"""
    Await.result(db.db.run(clearInputsTableQ.as[StrRes]), 30 seconds)
}
def resetJobs() = {
    val resetJobsQ = sql"""update jobs set inputid=4, stderr='',stdout='',owner='',started=NULL,ended=NULL,status='new';"""
    Await.result(db.db.run(resetJobsQ.as[StrRes]), 30 seconds)
}
def clearTable(tableName:String) = {
    val clear = sql"""delete from #$tableName ;"""
    Await.result(db.db.run(clear.as[StrRes]), 30 seconds)
}
def clearApks() = {
    
}
def clearResults(){
    val clearResults = sql"""delete from results; delete from resultdata;"""
    Await.result(db.db.run(clearResults.as[StrRes]), 30 seconds)
}
def clearAll(){
    clearInputs()
    clearResults()
    clearTable("jobs")
    clearTable("apks")
}

//clearResults()
//pauseJobs()
//clearAll()
//resumeJobs()
//Await.result(db.db.run(sql"""select count(*) from results;""".as[Count]), 30 seconds)

Upload needed files to database
-------------------------------

In [ ]:
clearAll()
setTimeout(600)

In [ ]:

val jarPath = File(s"${userhome}/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar")
val emptySpecPath = File(s"${userhome}/Documents/source/bounder/soot_hopper/specs/emptySpec.txt")
println(jarPath)
println(emptySpecPath)
// Upload empty spec run config and bounder jar
val emptyCfg = db.createConfig(
    emptySpecPath,
    jarPath,
    "Empty spec set"
)

// Upload data of APK files to database for experiments
def uploadApks() = {
    def appList = (expDir / ".." / benchmarkList).contentAsString.split("\n")
        .map{v => 
            val spl = v.split(":")
            val name = spl(0).replace(baseDir.toString,"")
            val path = spl(0)
            (name,path)
        }
    appList.foreach(a => db.uploadApk(a._1,File(a._2)))
}
uploadApks()

// push experiment .json files to database
def pushExp() = {
    val experiments = expDir.glob("**/*.cfg").toList
    println(experiments.size)
    experiments.foreach(v => db.createJob(v, Some("empty1"), emptyCfg))
}
pushExp()

Bulk Job Tasks
==============

In [ ]:
//db.downloadApk("/com.gpl.rpg.AndorsTrail/0.7.5/apk/com.gpl.rpg.AndorsTrail_46.apk", File("/Users/shawnmeier/Desktop/testDownload.apk"))
//clearAll()
//resumeJobs()
//clearResults()
//pauseJobs()